<a href="https://colab.research.google.com/github/Nehal0952/-Explainable-AI-labAssignment-/blob/main/Ass_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install pandas scikit-learn matplotlib seaborn shap lime

import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance
import shap
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
df = pd.read_csv("/content/bank-additional-full.csv.zip", sep=';')

# Preprocessing
df = pd.get_dummies(df, drop_first=True)
X = df.drop('y_yes', axis=1)
y = df['y_yes']

# Convert boolean columns to integers (0 or 1)
for col in X.columns:
    if X[col].dtype == 'bool':
        X[col] = X[col].astype(int)

# Identify and handle non-numeric columns in X
numeric_cols = X.select_dtypes(include=np.number).columns
X = X[numeric_cols]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Train model
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)
print("Accuracy:", accuracy_score(model.predict(X_test), y_test))

# 🔍 Permutation Importance
perm = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
sorted_idx = perm.importances_mean.argsort()[::-1]

plt.figure(figsize=(10,6))
sns.barplot(x=perm.importances_mean[sorted_idx], y=X.columns[sorted_idx])
plt.title("Permutation Importance")
plt.tight_layout()
plt.show()

# 🔍 SHAP Global + Local
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# Global summary plot
shap.summary_plot(shap_values, X_test)

# Force plot for one subscribed customer
# Find the index in X_test corresponding to the first subscribed customer
subscribed_original_idx = y_test[y_test == 1].index[0]
subscribed_shap_idx = X_test.index.get_loc(subscribed_original_idx)

shap.plots.force(shap_values[subscribed_shap_idx])

# 🔍 LIME Local Explanations
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X.columns.tolist(),
    class_names=['No', 'Yes'],
    mode='classification'
)

# One subscribed and one unsubscribed customer
subscribed_sample = X_test.loc[subscribed_original_idx]
unsubscribed_original_idx = y_test[y_test == 0].index[0]
unsubscribed_sample = X_test.loc[unsubscribed_original_idx]

# LIME explanation for subscribed
lime_subscribed = lime_explainer.explain_instance(
    data_row=subscribed_sample.values,
    predict_fn=model.predict_proba
)
lime_subscribed.show_in_notebook()

# LIME explanation for unsubscribed
lime_unsubscribed = lime_explainer.explain_instance(
    data_row=unsubscribed_sample.values,
    predict_fn=model.predict_proba
)
lime_unsubscribed.show_in_notebook()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=d8fab8aa7ac7b727d4965f7ac294c4cff2e1e9f3b3391fbb10750415271574d7
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


FileNotFoundError: [Errno 2] No such file or directory: '/content/bank-additional-full.csv.zip'